In [ ]:
# default_exp ppdd

# pico_pi_controller.ppdd
> System daemon

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
# export

from sys import byteorder
from os import getloadavg
from platform import node
from uptime import uptime
import threading, _thread
import argparse, logging
import pigpio, atexit
import datetime, time
import socketserver
from CircuitPython_pico_pi_common.codes import *

logger = logging.getLogger()
logging.basicConfig(level = logging.DEBUG)

In [ ]:
# export

empty    = bytearray(0)
pause    = bytearray(1)
hostname = bytearray(node(), "utf-8")
bosmang  = bytearray(int(0).to_bytes(1, byteorder))
#cmd_code = bytearray([122, 0x13, 0x03])  # FLICKER,0x13,3 seconds
cmd_code = bytearray([0x00])  

tcp_address = ('127.0.0.1', 16164)

REG_NAME = {}
REG_INT = []
for name, code in REG_CODE.items():
    REG_NAME[int.from_bytes(code, byteorder=byteorder)] = name
    REG_INT.append(int.from_bytes(code, byteorder=byteorder))
    
def log_txn(fname, message, msg=None):
        """Wrapper for logger."""
        id_str = ID_CODE.decode()
        i2c_str = '|'
        logger.info('%-4s %-7s %-47s %s' % (id_str, fname, message+str(msg or ''), i2c_str))

class PwrLedFlicker(threading.Thread):
    def __init__(self, duration):
        threading.Thread.__init__(self)
        self.duration = duration
        
    def run(self):
        try:
            with open("/sys/class/leds/led1/brightness", "w") as sys_pwr_led:
                for x in range(self.duration * 2):
                    sys_pwr_led.write("0")
                    sys_pwr_led.flush()
                    time.sleep(0.1)
                    sys_pwr_led.write("1")
                    sys_pwr_led.flush()
                    time.sleep(0.4)
            sys_pwr_led.close()
        except PermissionError:
            print("ppdd: must run as root.")
            _thread.interrupt_main()
            
class PPCcHandler(socketserver.StreamRequestHandler):
    def handle(self):
        fname='handle'
        global cmd_code
        log_txn(fname,"recvd request from {}".format(self.client_address[0]),
                " on {}".format(threading.current_thread().name))
        msg = self.rfile.readline().strip()
        log_txn(fname,"recvd from {}: ".format(self.client_address[0]),msg)
        cmd_code = msg
        self.wfile.write(
            bytearray([ord(c) for c in list('Command receivd. Awaiting confirmation from PPController')]))
        #self.wfile.write(empty)
        #self.finish()
        self.wfile.close()

@atexit.register
def goodbye():
    """Cancel pigpio event handler, close I2C peripheral & connection to pigpio"""
    global e, pi, ppdd_server
    fname='goodbye'
    if 'e' in globals() or 'pi' in globals():
        try:
            e.cancel()
            pi.bsc_i2c(0)  # Disable BSC peripheral
            pi.stop()
            log_txn(fname,"bsc_i2c cleanup completed.")
        except:
            log_txn(fname,"bsc_i2c error on cleanup.")
    if 'ppdd_server' in globals():
        try:
            ppdd_server.server_close()
            log_txn(fname,"TCP socket cleanup completed.")
        except:
            log_txn(fname,"TCP socket error on cleanup.")
    print(fname,"Exiting.")

def i2c_event_handler(id, tick):
    """Copied from local ppdd.py."""
    fname='i2c_event_handler'
    global pi, i2c_addr, flicker, cmd_code

    def reg_idf():
        fname='reg_idf'
        log_txn(fname, "sending identity: ", ID_CODE.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, ID_CODE)

    def reg_bos():
        fname='reg_bos'
        log_txn(fname, "sending bosmang status: ", str(bool(int.from_bytes(bosmang, byteorder=byteorder))))
        s, b, d = pi.bsc_i2c(i2c_addr, bosmang)

    def reg_tim():
        fname='reg_tim'
        dt = datetime.datetime.now()
        dt = dt.replace(
            tzinfo=datetime.timezone(datetime.timedelta(0), "UTC")
        )  # send local time converted from UTC of system time
        ts = dt.timestamp()
        log_txn(fname, "sending time.time(): ", int(ts) )
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(int(ts).to_bytes(4, byteorder)))

    def reg_cmd():
        fname='reg_cmd'
        global cmd_code
        log_txn(fname,"cmd_code: ", cmd_code )
        if cmd_code[0]:
            #log_txn(fname,"sending queud command: ", CMD_NAME[cmd_code[0]] )
            log_txn(fname,"sending queud command: ", cmd_code )
            s, b, d = pi.bsc_i2c(i2c_addr, cmd_code)
            # todo: wait for confirmation before clearing cmd_code
            cmd_code = bytearray([0x00])  

    def reg_hos():
        fname='reg_hos'
        log_txn(fname,
            "sending hostname: ", hostname.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, bytes([len(hostname)]) + hostname)

    def reg_lod():
        fname='reg_lod'
        load = bytearray("{:04.2f}".format(getloadavg()[0]), "utf-8")
        log_txn(fname, "sending loadavg: ", load.decode())
        s, b, d = pi.bsc_i2c(i2c_addr, load)

    def reg_upt():
        fname='reg_upt'
        # so the PPC can pause and wait for the long-ish uptime fetch
        s, b, d = pi.bsc_i2c(i2c_addr, pause)
        uptimeba = bytearray(int(uptime()).to_bytes(4, byteorder))
        log_txn(fname, "sending uptime: ", int.from_bytes(bytes(uptimeba), byteorder)
        )
        s, b, d = pi.bsc_i2c(i2c_addr, uptimeba)

    def reg_tzn():
        fname='reg_tzn'
        log_txn(fname, "sending time.timezone(): ", time.timezone, )
        s, b, d = pi.bsc_i2c(i2c_addr, bytearray(time.timezone.to_bytes(3, byteorder)))

    def reg_flk():
        fname='reg_flk'
        log_txn(fname, "Flickering PWR LED for "+str(d[1])+" seconds.")
        flicker = PwrLedFlicker(duration=d[1])
        flicker.start()

    def reg_clr():
        fname='reg_clr'
        #log_txn(fname, "CLR recieved, FIFO buffer to be emptied.")
        s, b, d = pi.bsc_i2c(i2c_addr, empty)

    s, b, d = pi.bsc_i2c(i2c_addr)
    if b:
        #log_txn(fname, "Register probe recvd, status:",s)
        if d[0] in REG_INT:
            reg_handler = locals()['reg_'+REG_NAME[d[0]].lower()]
            log_txn(fname, "calling register handler: reg_"+REG_NAME[d[0]].lower())
            reg_handler()


def main():
    fname='main'
    global e, pi, ppdd_server, i2c_addr, flicker, bosmang
    
    # flicker power LED on startup, checking for root access
    flicker = PwrLedFlicker(duration=2)
    flicker.start()
    time.sleep(0.1)
    
    # setup pigpio, checking that it's running
    pi = pigpio.pi()
    if not pi.connected:
        log_txn(fname,"pigpiod not running. Exiting.")
        exit()

    # handle command-line arguments
    parser = argparse.ArgumentParser(description='ppdd argument parser')
    parser.add_argument("-a", default='0x13', type=str, help="Default I2C address: 0x13")
    parser.add_argument("-b", action="store_true", help="Set bosmang status to True")
    
    args, unknown = parser.parse_known_args()
    
    if unknown:
        log_txn(fname,"unrecognized command line arguments: ",unknown)
    
    i2c_addr = int(args.a,0)
    
    if args.b:
        bosmang = bytearray(int(1).to_bytes(1, byteorder))
        
    log_txn(fname,"Initializing I2C peripheral on address: ", hex(i2c_addr))
    log_txn(fname,"  bosmang:  ", str(bool(int.from_bytes(bosmang, byteorder=byteorder))))
    log_txn(fname,"  hostname: ", hostname.decode())

    # setup event handler for incoming I2C messages from PPController
    e = pi.event_callback(pigpio.EVENT_BSC, i2c_event_handler)
    pi.bsc_i2c(i2c_addr)

    # setup socket server for incoming commands from ppcc
    ppdd_server = socketserver.ThreadingTCPServer(tcp_address, PPCcHandler)
    ppdd_server.serve_forever()
    log_txn(fname,"Listening for ppcc commands on port ", tcp_address[1])
        
    while True:
        time.sleep(600)
        
if __name__ == '__main__':
    try:
        main()
    except KeyboardInterrupt as e:
        exit(str('Exiting.'))

ppdd: must run as root.


In [ ]:
%tb

No traceback available to show.


In [ ]:
# hide
try:
    from IPython.display import display, Javascript
    display(Javascript('IPython.notebook.save_checkpoint();'))
    from time import sleep
    sleep(0.3)
    from nbdev.export import notebook2script
    notebook2script()
except ModuleNotFoundError:
    pass
    """CircuitPython kernel has no nbdev"""

<IPython.core.display.Javascript object>

Converted 00_core.ipynb.
Converted 50_ppdd.ipynb.
Converted 60_ppcc.ipynb.
Converted index.ipynb.
